# Bonus

In the real world, debugging agents often need to be able to read logs from another system. Let's give the agent ability to read logs from a database instead. 

**Be sure to restart the kernel before proceeding**

Let's create a job 

In [ ]:
%cd /home/jovyan/bonus

In [ ]:
!unzip logs.zip

In [ ]:
import asyncio
import boto3
from datetime import timedelta
from langchain_aws import ChatBedrock, ChatBedrockConverse
from langchain_core.runnables import RunnableConfig
from langchain_core.tools import BaseTool
from langchain_mcp_adapters.sessions import StreamableHttpConnection
from langgraph.prebuilt import create_react_agent
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_core.messages import HumanMessage, SystemMessage
from utils import print_agent_response, execute_query
from phoenix.otel import register
import os


tracer_provider = register(
  project_name="default", # Default is 'default'
  auto_instrument=True,
  endpoint =  "http://phoenix-0.phoenix.phoenix.svc.cluster.local:6006/v1/traces"
)


In [ ]:
# limit the number of times an agent runs through LLM to 10
recursion_limit = 30
model = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
# We are using Claude hosted on Bedrock, so let's create a Bedrock session
bedrock_client = boto3.client("bedrock-runtime", region_name="us-west-2")

# Use the session to define a LLM that will drive the Agent
llm = ChatBedrockConverse(
    model=model,
    temperature=0,
    max_tokens=None,
    client=bedrock_client,
)

In [ ]:
# Define a system prompt. Let the LLM know what its role is and guidelines
system_prompt="""
You are Sparky McSparkface, a specialized Apache Spark troubleshooting assistant designed for data engineering teams.
Your purpose is to diagnose, analyze, and resolve Spark application issues by systematically examining logs, metrics, and application data.

Core Capabilities:
- Troubleshoot Spark job failures and identify root causes
- Recommend performance optimizations for Spark applications
- Analyze resource utilization patterns and bottlenecks
- Provide technical, evidence-based explanations tailored to various expertise levels

Output Format:
Structure your responses as:

## Summary
Brief overview of the primary issue and severity level (Critical/High/Medium/Low)

## Findings
For each issue identified:
- **Issue**: Clear description
- **Evidence**: Specific log excerpts, metrics, or configuration values
- **Impact**: How this affects performance/functionality
- **Recommendation**: Actionable solution with configuration changes

## Next Steps
Prioritized list of actions to take

Evidence Requirements:
Only include findings supported by concrete evidence such as log excerpts with timestamps, specific metric values, configuration settings, or resource utilization data.

If you cannot find supporting evidence for a potential issue, exclude it from your response.
If you lack sufficient information for confident assessment, state: "I don't have enough information to confidently assess this."
"""


agent = create_react_agent(
    model=llm,
    tools=[execute_query],
    prompt=SystemMessage(system_prompt)
)

# Ready?

Let's see if it can tell us why the Spark application failed. It has access to the logs, it can tell us what's wrong with our app right?

In [ ]:
result = await agent.ainvoke(
    input={"messages": [HumanMessage(content="hi, tell me why spark-2435f7978c3146baacc286c789bed535 failed?")]},
    config=RunnableConfig(recursion_limit=recursion_limit),
)

print_agent_response(result)

# Uh oh

You likely gotten a big stack of errors after waiting a minute. What happened?

Head over to your tracing endpoint and see if you can tell what's going on.


## What likely happened

LLMs are quite good at creating SQL queries. However, this LLM was not given enough information about the table such as the schema. So it likely went through the trial and error process to find the right schema, right query, and right context amount.


## How can you fix it?

Here's the schema for the table. Can you think of a way to fix this?


```
• id (INTEGER, PRIMARY KEY, AUTOINCREMENT) - Unique identifier for each log entry
• spark_app_id (TEXT) - Application identifier from the JSON data (e.g., "app-12345")
• log_message (TEXT) - The actual log message content, including multi-line stack traces
• log_level (TEXT) - Log severity level (e.g., "ERROR", "INFO", "WARN")
• time (TIMESTAMP) - Parsed timestamp from log entries in format YY/MM/DD HH:MM:SS
```

# Potential fixes

<details>
<summary><strong>Solution 1</strong></summary>

Provide the schmea information in the system promopt or the function comment. For example:

```
you have access to a SQLite table that contains log messages from failed applications. The schema of the table is as follows:
• id (INTEGER, PRIMARY KEY, AUTOINCREMENT) - Unique identifier for each log entry
• spark_app_id (TEXT) - Application identifier from the JSON data (e.g., "app-12345")
• log_message (TEXT) - The actual log message content, including multi-line stack traces
• log_level (TEXT) - Log severity level (e.g., "ERROR", "INFO", "WARN")
• time (TIMESTAMP) - Parsed timestamp from log entries in format YY/MM/DD HH:MM:SS

Because the number of rows returned by a query could be large, be sure to limit your query to 10 - 20 and select the log_message column only.
```

</details>

<details>
<summary><strong>Solution 2</strong></summary>

Create a dedicated tool that retrieves data from the table without the LLM generating SQL queries. For example:

```
@tool
def get_logs_by_app_id(spark_app_id, limit=10):
   """
   Get logs for a specific Spark application ID.

   Args:
       spark_app_id (str): The Spark application ID to filter by
       limit (int): Maximum number of rows to return (default: 10)

   Returns:
       list: List of tuples containing (id, spark_app_id, log_message, log_level, time)
   """
   cursor = conn.cursor()
   cursor.execute(
       "SELECT  FROM logs WHERE sparkapp_id = ? ORDER BY time DESC LIMIT ?",
       (spark_app_id, limit)
   )
   return cursor.fetchall()
```
</details>
